# Classification sur MNIST avec réseaux de neurones convolutionels (CNN)

Dans ce cahier, nous allons utiliser les CNN pour effectuer une tache de classification sur le jeu de données MNIST. Nous avons déjà effectué cette tâche sur MNIST dans la cahier precedent en utilisant un reseaux multicouche dense. Nous allons donc voir ici a quel point les CNN apportent une amelioration dans les taches visuel.


## Charger les data

Comme dans le carnet précédent, nous commençons par
- chargement et normalisation du jeu de données
- effectuer le fractionnement habituel dans un ensemble de formation / test
- transformer les étiquettes en vecteurs de type à codage unique.

In [ ]:
from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import RMSprop
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline 

In [ ]:
# Download the data, shuffled and split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# and let us look at the numbers
print("These are numbers corresponding to ",y_train[0]," ",y_train[1]," ",y_train[2]," and ",y_train[3])
plt.subplot(221)
plt.imshow(x_train[0], cmap=plt.get_cmap('gray'))
plt.subplot(222)
plt.imshow(x_train[1], cmap=plt.get_cmap('gray'))
plt.subplot(223)
plt.imshow(x_train[2], cmap=plt.get_cmap('gray'))
plt.subplot(224)
plt.imshow(x_train[3], cmap=plt.get_cmap('gray'))

In [ ]:
# convert class vectors to binary class matrices
num_classes = 10
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

print("The cateragories of these image is now encoded as : \n",y_train[0][:],"\n",y_train[1][:],"\n",y_train[2][:],"\n",y_train[3][:],"\n" )

**ATTENTION !!!!!**

Contrairement a la lecon precedence, nous n'allons  **PAS** transformer les images en vecteurs unidimensionnels. Nous voulons **au contraire** les conserver en tant qu’images bidimensionnelles. En fait, pour que keras et tensorflow fonctionnent correctement, nous allons les transformer de matrices $ 28 * 28 $ en tenseurs  $ 28 * 28 * 1 $ !

In [ ]:
img_rows, img_cols = 28, 28
input_shape = (img_rows, img_cols, 1)
x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

x_train.shape

## Créer un CNN

Nous allons maintenant créer un réseau de convolution. Les deux premières couches auront chacune $ 32 $ filtres convolutifs de la taille du noyau $ 3 \ times 3 $. Ensuite, nous ajoutons une couche de regroupement qui prendra $ 2x2 $ carrés et en produira le maximum (maxpooling) et transformera "l'image" résultante en un vecteur (la couche d'aplatissement: Flatten layer). Enfin, nous ajouterons un perceptron multicouche. Avec les couches abandonnées, cela donne:

In [ ]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.summary()

model.compile(loss="categorical_crossentropy",
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

Commentons le nombre de paramètres et l'espace de sortie dans chaque couche, ce qui aidera à comprendre la structure des couches.

- La première couche de Conv2D a un noyau de taille $ 3 \times 3 $, qui a $ 3 * 3 + 1 = 10 $ paramètres (n'oubliez pas le biais!). Il existe 32 filtres de ce type, d’où un total de $ 320 $ paramètres. Chaque filtre est ensuite appliqué indépendamment à chaque région de l'image d'origine: il y a 26 régions fois, 26 en sortie, 32 filtres; la sortie a donc la forme $ (26, 26, 32) $.

- La deuxième couche de Conv2D a aussi un noyau de taille $ 3 \times 3 $, mais agit maintenant sur un volume de profondeur 32. Ainsi, le nombre de paramètres de chaque noyau est $ 3 * 3 * 32 + 1 = 289 $. Il y a encore 32 filtres, conduisant à $ 289 * 32 = 9248 $ paramètres. Chaque filtre est appliqué à l’une des régions possibles $ 24 \times 24 $. La forme de sortie est donc $ (24, 24, 32) $.

- La couche max_pool n'a pas de paramètres libres. Il prend simplement en sortie le maximum de l'entrée dans les régions disjointes de taille $ 2 \times 2 $, indépendamment pour chaque profondeur. La sortie est $ (12, 12, 32) $.

- Enfin, nous appliquons un abandon et un aplatissement conduisant à un seul vecteur de taille $ 12 * 12 * 32 = 4608 $, que nous utilisons comme entrée pour un MLP.
Nous allons maintenant créer un réseau de convolution. Les deux premières couches auront chacune $ 32 $ filtres convolutifs de la taille du noyau $ 3 \times 3 $. Ensuite, nous ajoutons une couche de regroupement qui prendra $ 2x2 $ carrés et en produira le maximum (la couche de maximum de covoiturage) et transformera "l'image" résultante en un vecteur (la couche d'aplatissement). Enfin, nous ajouterons un perceptron multicouche. Avec les couches abandonnées, cela donne:

## Entrainement du CNN

Laissez-nous enfin former le réseau. Ce sera cependant assez lent. Chaque époque prend environ $ 10 $ secondes sur un processeur graphique haut de gamme tel que NVIDIA $ K80 $, mais prend de une à trois minutes sur un un ordinateur de bureau standard.

In [ ]:
batch_size = 128
epochs = 12

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Malgré tout, le résultat est gratifiant, sans trop de travail, nous avons atteint une erreur en deçà du pour cent! Si vous êtes patient (ou que vous avez un bon processeur graphique), vous pouvez essayer de jouer avec les paramètres et d’atteindre une performance proche de $ 0.6 \% $ error.

*Exercice:* MNIS est ttrop facile. En remplacent quelques lignes (**from keras.datasets import fashion_mnist**, puis  **((trainX, trainY), (testX, testY)) = fashion_mnist.load_data()**) on pourra utiliser le datase fashion_mnist. Jouer avec les architectures pour obtenir le meilleur resultat possible!
